# **Quora Insincere Questions**

### **Giới thiệu:**
​
Quora là một nền tảng cho phép mọi người học hỏi lẫn nhau. Trên Quora, mọi người có thể đặt câu hỏi và kết nối với những người khác, những người đưa ra những thông tin bổ ích và câu trả lời chất lượng. Có một thách thức là loại bỏ những câu hỏi thiếu chân thành - những câu hỏi được đặt ra dựa trên ý đồ sai hoặc có ý định đưa ra một tuyên bố hơn là tìm những câu trả lời hữu ích.
​

Bài toán: Cho một bộ dữ liệu gồm những câu hỏi chân thành (sincere questions) và những câu hỏi không chân thành (insincere questions), cần xác định và đánh dấu những câu hỏi không chân thành.
​

In [ ]:
import os
import json
import string
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from wordcloud import WordCloud, STOPWORDS
from collections import defaultdict #Tương tự như python dictionary nhưng sẽ tự động sinh ra một default key khi truy cập key không tồn tại 

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

## **1. Phân tích dữ liệu** 

In [ ]:
train_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
test_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.shape

Bộ dữ liệu train có tổng cộng 1306122 hàng và 3 cột


In [ ]:
train_df.info()

Dữ liệu không có giá trị null.

Tập dữ liệu train gồm 2 cột kiểu text (qid và question_text) và 1 cột kiểu số (target):

* qid: ID của câu hỏi.
* question_text: Nội dung của câu hỏi.
* target: insincere = 1, sincere = 0

In [ ]:
val_counts = train_df["target"].value_counts()
val_counts

Có 1225312 câu hỏi chân thành và 80810 câu hỏi không chân thành

In [ ]:
sincere_q = val_counts[0]/val_counts.sum()
sincere_q = sincere_q*100
print('Có {}% câu hỏi là chân thành còn phần còn lại là không chân thành '.format(sincere_q))

### **Trực quan hóa dữ liệu**

In [ ]:
trace = go.Bar(
    x=val_counts.index,
    y=val_counts.values,
    marker=dict(
        color=val_counts.values,
        colorscale = 'Picnic',
        reversescale = True
    ),
)

layout = go.Layout(
    title='Lượng câu hỏi mỗi loại',
    font=dict(size=14)
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="TargetCount")

labels = (np.array(val_counts.index))
sizes = (np.array((val_counts / val_counts.sum())*100))

trace = go.Pie(labels=labels, values=sizes)
layout = go.Layout(
    title='Phân phối loại câu hỏi',
    font=dict(size=14),
    width=500,
    height=500,
)
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="usertype")

Dựa vào tỉ lệ và biểu đồ ta thấy dữ liệu không cân bằng vì lượng câu hỏi chân thành lớn hơn nhiều so với lượng câu hỏi không chân thành.

=> Sẽ so sánh kết quả model khi dữ liệu không cân bằng với model khi dữ liệu đã được resampling

### **Word Cloud**

Sử dụng thư viện WordCloud sẽ cho ta thấy tần suất xuất hiện các từ trong câu hỏi. Từ nào xuất hiện nhiều kích thước sẽ càng lớn và ngược lại.

In [ ]:
from wordcloud import WordCloud

In [ ]:
insincere_wordcloud = WordCloud(width=5000, 
                                height=4000,
                                colormap='Reds',
                                background_color ='white', 
                                min_font_size = 8).generate(str(train_df[train_df["target"] == 1]["question_text"]))
plt.figure(figsize=(15,10), facecolor=None)
plt.imshow(insincere_wordcloud)
plt.axis("off")
plt.show();

Các từ có tần suất xuất hiện nhiều trong câu hỏi không chân thành

In [ ]:
sincere_wordcloud = WordCloud(width=5000, 
                                height=4000,
                                colormap='Greens',
                                background_color ='white', 
                                min_font_size = 8).generate(str(train_df[train_df["target"] == 0]["question_text"]))
plt.figure(figsize=(15,10), facecolor=None)
plt.imshow(sincere_wordcloud)
plt.axis("off")
plt.show();

Các từ có tần suất xuất hiện nhiều trong câu hỏi chân thành


## **2. Tiền xử lý dữ liệu** 

**Tiền xử lý dữ liệu là một bước rất quan trọng trong việc giải quyết bất kỳ vấn đề nào trong lĩnh vực Học Máy. Hầu hết các bộ dữ liệu được sử dụng trong các vấn đề liên quan đến Học Máy cần được xử lý, làm sạch và biến đổi trước khi một mô hình có thể được huấn luyện trên những bộ dữ liệu này. Các kỹ thuật tiền xử lý dữ liệu được sử dụng trong bài:**
* Xóa các stopwords: Stopwords là các từ có ít hoặc không có ý nghĩa gì đặc biệt khi xây dựng các đặc trưng. Đây thường là giới từ, trợ từ có tần suất xuất hiện tương đối cao trong một văn bản thông thường ví dụ như: a, an, the... Thư viện nltk có một danh sách các stopword có sẵn
* Xử lý từ gốc và ngữ pháp: Trong các ngữ cảnh khác nhau, các từ gốc thường được gắn thêm các tiền tố và hậu tố vào để đúng với ngữ pháp. Ví dụ các từ: WATCHES, WATCHING, and WATCHED. Chúng ta có thể thấy rằng chúng đều có chung từ gốc là WATCH
* Bên cạnh đó em cũng dùng tokenization, xóa bỏ các khoảng trắng thừa, chuẩn hóa chữ cái viết hoa, xử lý các kí tự số

In [ ]:
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk_stopwords = stopwords.words('english')

wordnet_lemmatizer = WordNetLemmatizer()

def lemSentence(sentence):
    token_words = word_tokenize(sentence) #Tokenize 
    lem_sentence = []
    for word in token_words:
        lem_sentence.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        lem_sentence.append(" ")
    return "".join(lem_sentence)

def clean(message, lem=True):
    # Loại bỏ dấu câu
    message = message.translate(str.maketrans('', '', string.punctuation))
    
    # Loại bỏ chữ số
    message = message.translate(str.maketrans('', '', string.digits))
    
    # Loại bỏ stopwords
    message = [word for word in word_tokenize(message) if not word.lower() in nltk_stopwords]
    message = ' '.join(message)
    
    # Xử lý từ gốc và ngữ pháp
    if lem:
        message = lemSentence(message)
    
    return message

In [ ]:
train_df['question_text_cleaned'] = train_df.question_text.apply(lambda x: clean(x, True))

In [ ]:
test_df['question_text_cleaned'] = test_df.question_text.apply(lambda x: clean(x, True))

## **3. Mô hình huấn luyện**

Chia tập dữ liệu thành 2 phần theo tỉ lệ 80-20:
* Tập train dùng để huấn luyện model
* Tập validation dùng để đánh giá model

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_df['question_text_cleaned'], train_df.target, test_size=0.2, stratify = train_df.target.values)

In [ ]:
# from sklearn.model_selection import GridSearchCV, StratifiedKFold

# #Setting the range for class weights
# weights = np.linspace(0.0,0.99,200)

In [ ]:
# Tạo dictionary cho grid search
# param_grid = {'class_weight': [{0:x, 1:1.0-x} for x in weights]}

In [ ]:
#Fitting grid search vào train data với 5 folds
# lr = LogisticRegression(solver='saga')

# gridsearch = GridSearchCV(estimator= lr, 
#                           param_grid= param_grid,
#                           cv=StratifiedKFold(), 
#                           n_jobs=-1, 
#                           scoring='f1', 
#                           verbose=2).fit(X_train, y_train)

### **Công cụ triển khai**
**CountVectorizer**

Sau khi đã làm sạch dữ liệu văn bản, những từ này cần được mã hóa dưới dạng số để sử dụng vào trong thuật toán học máy. Quá trình này được gọi là trích xuất đặc trưng (vectơ hóa). CountVectorizer được sử dụng để chuyển đổi một bộ các tài liệu văn bản thành vectơ và đọc hiểu dữ liệu dựa vào tần số xuất hiện từ vựng đó.

Nhược điểm:
* Không xác định được mức độ quan trọng giữa các từ.
* Không xác định mối quan hệ giữa các từ với nhau (VD: sự tương đồng về mặt ngữ nghĩa)
* Chỉ đánh giá những từ xuất hiện nhiều trong kho văn bản mới có ý nghĩa về mặt thống kê.

**Logistic Regression**

Logistic Regression là 1 thuật toán phân loại được dùng để gán các đối tượng cho 1 tập hợp giá trị rời rạc (như 0, 1, 2, ...). Một ví dụ điển hình là phân loại email, gồm có email công việc, email gia đình, email spam, ...
=> Mô hình thích hợp cho bài toán

**Pipeline**

* Một pipeline trong sklearn là một tập các chuỗi thuật toán để trích xuất đặc trưng, tiền xử lý, chuyển hóa và huấn luyện dữ liệu sử dụng các thuật toán học máy cụ thể. 
* Mỗi pipeline bao gồm một vài bước nhất định, mỗi bước bao gồm một tham số là tên của bước, tham số còn lại là bộ chuyển đổi dữ liệu tương ứng (thường gọi là transformer). Bước cuối cùng trong một pipeline được gọi là estimator. Một estimator có thể là một thuật toán phân lớp, một thuật toán hồi quy, một mạng nơ-ron hay có thể là một thuật toán học máy không giám sát.
* Để huấn luyện estimator tại bước cuối cùng của pipeline, ta phải gọi phương thức fit củapipeline và cung cấp dữ liệu để huấn luyện.
* Một khi dữ liệu đã được huấn luyện bằng cách sử dụng estimator trong pipeline, ta có thể sử dụng pipeline đó để dự đoán đầu ra cho dữ liệu mới bằng cách sử dụng phương thức predict.





In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

pipeline_weight_cv = Pipeline([("cv", CountVectorizer(analyzer="word", ngram_range=(1,2), max_df=0.9)),
                     ("clf", LogisticRegression(solver="saga", class_weight={0: 0.0619, 1: 0.9381}, max_iter=10000, verbose=1, n_jobs=-1))])


Giá trị class_weight: Phạt các mẫu ở class[i] với class_weight[i]. Nghĩa là class_weight có giá trị càng lớn thì sẽ càng nhấn mạnh vào class đó.

Thay đổi class_weight: Vì sự mất cân bằng trong bộ dữ liệu (số lượng câu hỏi chân thành (0) quá nhiều (93.81%) so với số lượng câu hỏi không chân thành (1) (6.19%)) => Tăng giá trị class_weight vào 1 và giảm giá trị class_weight vào 0. 

In [ ]:
def result(y_test, y_pred):
    print("Accuracy: ", accuracy_score(y_test, y_pred))
    print("F1-score: ",f1_score(y_test, y_pred, pos_label=1))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm/np.sum(cm), annot=True, 
            fmt='.2%', cmap='YlGnBu')

In [ ]:
lr_model_weight_cv = pipeline_weight_cv.fit(X_train, y_train)

In [ ]:
y_pred_weight_cv = lr_model_weight_cv.predict(X_val)

In [ ]:
result(y_val, y_pred_weight_cv)

Do sự mất cân bằng trong dữ liệu => Xảy ra hiện tượng overfitting khi có sự chênh lệch lớn giữa F1-score và Accuracy

In [ ]:
pipeline_balanced_cv = Pipeline([("cv", CountVectorizer(analyzer="word", ngram_range=(1,2), max_df=0.9)),
                     ("clf", LogisticRegression(solver="saga", class_weight="balanced", max_iter=10000, verbose=1, n_jobs=-1))])


Để giá trị class_weight = "balanced": Sử dụng tổng lượng mẫu để tự động điều chỉnh trọng số sao cho tỉ lệ nghịch với số mẫu của các lớp.

In [ ]:
lr_model_balanced_cv = pipeline_balanced_cv.fit(X_train, y_train)

In [ ]:
y_pred_balanced_cv = lr_model_balanced_cv.predict(X_val)

In [ ]:
result(y_val, y_pred_balanced_cv)

### **Downsampling**

Down sampling là việc giảm số lượng các mẫu của nhóm đa số để nó trở nên cân bằng với số mẫu của nhóm thiểu số. 
* Ưu điểm: Làm cân bằng mẫu một cách nhanh chóng, dễ dàng tiến hành thực hiện mà không cần đến thuật toán giả lập mẫu.
* Nhược điểm: Kích thước mẫu sẽ bị giảm đáng kể.

In [ ]:
insincere = train_df[train_df['target'] == 0]
sincere = train_df[train_df['target'] == 1]

Đảo câu hỏi chân thành = 1, câu hỏi không chân thành = 0 để tiến hành down sampling.

In [ ]:
len(sincere)

Lấy số lượng câu hỏi không chân thành, từ đó giảm số lượng câu hỏi chân thành xuống bằng với số lượng câu hỏi không chân thành.

In [ ]:
insincere_batch = insincere[:len(sincere)]

In [ ]:
insincere_batch

In [ ]:
test_batch = pd.concat([insincere_batch, sincere])

Ghép lượng câu hỏi chân thành và không chân thành vào với nhau.

In [ ]:
test_batch

In [ ]:
from sklearn.model_selection import train_test_split

X_train_down, X_val_down, y_train_down, y_val_down = train_test_split(test_batch['question_text_cleaned'], test_batch.target, test_size=0.2, stratify = test_batch.target.values)

**Fine tuning giá trị C**

Giá trị C: Chính quy hóa (Regularization): Điều chỉnh số lỗi bằng cách sử dụng hàm hợp lý trên tập training và tránh overfitting trong khi vẫn giữ được tính tổng quát của nó.

Thay đổi các giá trị C để tìm được model phù hợp.

In [ ]:
C_param_range = np.arange(0.1, 0.501, 0.05)

train_acc_table_cv = pd.DataFrame(columns = ['C_parameter','Accuracy', 'F1-Score'])
train_acc_table_cv['C_parameter'] = C_param_range

j = 0
for i in C_param_range:
    
    pipeline_down_C_cv = Pipeline([("cv", CountVectorizer(analyzer="word", ngram_range=(1,2), max_df=0.9)),
                     ("clf", LogisticRegression(solver="saga", class_weight="balanced", C=i, max_iter=10000, verbose=1, n_jobs=-1))])

    lr_down_C_cv = pipeline_down_C_cv.fit(X_train_down, y_train_down)
    
    y_pred_down_C_cv = lr_down_C_cv.predict(X_val_down)
    
    train_acc_table_cv.iloc[j,1] = accuracy_score(y_val_down, y_pred_down_C_cv)
    train_acc_table_cv.iloc[j,2] = f1_score(y_val_down, y_pred_down_C_cv)

    j += 1
       

In [ ]:
train_acc_table_cv

Chọn giá trị C = 0.45 có kết quả Accuracy và F1-score tốt nhất

In [ ]:
pipeline_down_best_cv = Pipeline([("cv", CountVectorizer(analyzer="word", ngram_range=(1,2), max_df=0.9)),
                     ("clf", LogisticRegression(solver="saga", class_weight="balanced", C=0.45, max_iter=10000, verbose=1, n_jobs=-1))])


In [ ]:
lr_down_best_cv = pipeline_down_best_cv.fit(X_train_down, y_train_down)

In [ ]:
y_pred_down_best_cv = lr_down_best_cv.predict(X_val_down)

In [ ]:
result(y_val_down, y_pred_down_best_cv)

**TF-IDF vectorize**

TF-IDF là viết tắt từ cụm từ tiếng Anh: term frequency–inverse document frequency, là một thống kê số học nhằm phản ánh tầm quan trọng của một từ đối với một văn bản trong một tập hợp hay một ngữ liệu văn bản. TF–IDF thường dùng dưới dạng là một trọng số trong tìm kiếm truy xuất thông tin, khai thác văn bản, và mô hình hóa người dùng.

TF-IDF được đánh giá là tốt hơn CountVectorizer vì ngoài tập trung vào tần số xuất hiện của một từ, nó còn cho ra độ quan trọng của từ đó trong kho văn bản => Loại bỏ những từ ít quan trọng, model sẽ bớt phức tạp

Weight

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

pipeline_weight_tfidf = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", ngram_range=(1,2), max_df=0.9)),
                     ("clf", LogisticRegression(solver="saga", class_weight={0: 0.0619, 1: 0.9381}, max_iter=10000, verbose=1, n_jobs=-1))])


In [ ]:
lr_model_weight_tfidf = pipeline_weight_tfidf.fit(X_train, y_train)

In [ ]:
y_pred_weight_tfidf = lr_model_weight_tfidf.predict(X_val)

In [ ]:
result(y_val, y_pred_weight_tfidf)

Balanced

In [ ]:
pipeline_balanced_tfidf = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", ngram_range=(1,2), max_df=0.9)),
                     ("clf", LogisticRegression(solver="saga", class_weight="balanced", max_iter=10000, verbose=1, n_jobs=-1))])


In [ ]:
# lr_model_balanced_tfidf = pipeline_balanced_tfidf.fit(X_train, y_train)

In [ ]:
# y_pred_balanced_tfidf = lr_model_balanced_tfidf.predict(X_val)

In [ ]:
# result(y_val, y_pred_balanced_tfidf)

Fine tuning C

In [ ]:
C_param_range = np.arange(0.1, 0.501, 0.05)

train_acc_table_tfidf = pd.DataFrame(columns = ['C_parameter','Accuracy', 'F1-Score'])
train_acc_table_tfidf['C_parameter'] = C_param_range

j = 0
for i in C_param_range:
    
    pipeline_down_C_tfidf = Pipeline([("cv", CountVectorizer(analyzer="word", ngram_range=(1,2), max_df=0.9)),
                     ("clf", LogisticRegression(solver="saga", class_weight="balanced", C=i, max_iter=10000, verbose=1, n_jobs=-1))])

    lr_down_C_tfidf = pipeline_down_C_tfidf.fit(X_train_down, y_train_down)
    
    y_pred_down_C_tfidf = lr_down_C_tfidf.predict(X_val_down)
    
    train_acc_table_tfidf.iloc[j,1] = accuracy_score(y_val_down, y_pred_down_C_tfidf)
    train_acc_table_tfidf.iloc[j,2] = f1_score(y_val_down, y_pred_down_C_tfidf)

    j += 1

In [ ]:
train_acc_table_tfidf

Chọn C = 0.5 cho ra kết quả Accuracy và F1-score tốt nhất.

In [ ]:
pipeline_down_best_tfidf = Pipeline([("cv", CountVectorizer(analyzer="word", ngram_range=(1,2), max_df=0.9)),
                     ("clf", LogisticRegression(solver="saga", class_weight="balanced", C=0.5, max_iter=10000, verbose=1, n_jobs=-1))])


In [ ]:
lr_down_best_tfidf = pipeline_down_best_tfidf.fit(X_train_down, y_train_down)

In [ ]:
y_pred_down_best_tfidf = lr_down_best_tfidf.predict(X_val_down)

In [ ]:
result(y_val_down, y_pred_down_best_tfidf)

## **4. Submission**

In [ ]:
test_df['prediction'] = lr_down_best_tfidf.predict(test_df['question_text_cleaned'])

In [ ]:
results = test_df[['qid','prediction']]
results.to_csv('submission.csv', index=False)

In [ ]:
results.head(5)

### **Hướng cải thiện**
* Tìm cách fine tuning model
* Sử dụng model khác